In [ ]:
import numpy as np
from scipy.sparse import csc_matrix, identity, kron
from scipy.sparse.linalg import eigsh
import time

# --- Define 2x2 Pauli Matrices (sparse) ---
pauli_I = csc_matrix(np.eye(2))
pauli_X = csc_matrix(np.array([[0, 1], [1, 0]]))
pauli_Z = csc_matrix(np.array([[1, 0], [0, -1]]))


In [ ]:

# --- Helper function to build N-qubit operators ---

def get_pauli_prod(N, pauli_map):
    """
    Builds an N-qubit operator as a tensor product of Pauli matrices.

    Args:
        N (int): Number of qubits.
        pauli_map (dict): A dictionary {index: pauli_str} where index is the
                          qubit index (0 to N-1) and pauli_str is 'X', 'Y', 'Z'.
                          Qubits not in the map are assumed to be Identity.

    Returns:
        scipy.sparse.csc_matrix: The N-qubit operator.
    """
    pauli_dict = {'X': pauli_X, 'Z': pauli_Z, 'I': pauli_I}

    # Create a list of 2x2 operators
    ops_list = [pauli_I] * N
    for i, p_str in pauli_map.items():
        if i < N:
            ops_list[i] = pauli_dict.get(p_str, pauli_I)

    # Compute the tensor product
    H = ops_list[0]
    for op in ops_list[1:]:
        H = kron(H, op, format='csc')
    return H


In [ ]:

# --- Functions to build the h_i operators ---

def build_h_operators(N):
    """
    Builds the four h_i operators for an N-qubit system.

    Args:
        N (int): Number of qubits.

    Returns:
        list: A list [h1, h2, h3, h4] of sparse matrices.
    """
    D_N = 2**N

    # h1 = sum_{i=0}^{N-2} Z_i Z_{i+1}
    h1 = csc_matrix((D_N, D_N))
    for i in range(N - 1):
        h1 += get_pauli_prod(N, {i: 'Z', i + 1: 'Z'})

    # h2 = sum_{i=0}^{N-1} Z_i
    h2 = csc_matrix((D_N, D_N))
    for i in range(N):
        h2 += get_pauli_prod(N, {i: 'Z'})

    # h3 = sum_{odd sites} X_i  (i.e., i = 0, 2, 4, ...)
    h3 = csc_matrix((D_N, D_N))
    for i in range(0, N, 2):
        h3 += get_pauli_prod(N, {i: 'X'})

    # h4 = sum_{even sites} X_i (i.e., i = 1, 3, 5, ...)
    h4 = csc_matrix((D_N, D_N))
    for i in range(1, N, 2):
        h4 += get_pauli_prod(N, {i: 'X'})

    return [h1, h2, h3, h4]


In [ ]:

# --- Main function to compute the gap ---

def compute_hamiltonian_gap(N, sigma_weights, k_param):
    """
    Computes the energy gap (E1 - E0) of the effective Hamiltonian.

    H = sum_{i=1}^{4} (k * sigma_{i,c})^2 * (h_i tensor I - I tensor h_i)^2

    Args:
        N (int): Number of qubits (e.g., 4, 6, 8).
        sigma_weights (list or np.array): List of the 4 coefficients [sigma_{1,c}, sigma_{2,c}, ...].
        k_param (float): The replica parameter 'k'.

    Returns:
        float: The energy gap (E1 - E0).
    """
    if len(sigma_weights) != 4:
        raise ValueError("sigma_weights must be a list of 4 numbers.")

    print(f"Starting calculation for N = {N} qubits.")
    print(f"Total Hilbert space dimension = 2^(2*N) = 2^{2*N} = {2**(2*N)}")

    # --- 1. Build h_i operators ---
    start_time = time.time()
    h_ops = build_h_operators(N)
    print(f"Built h_i operators in {time.time() - start_time:.4f} sec.")

    # --- 2. Build the full Hamiltonian H ---
    start_time = time.time()
    D_N = 2**N
    D_total = 2**(2 * N)

    # N-qubit Identity
    I_N = identity(D_N, format='csc')

    # Total Hamiltonian (starts as zero matrix)
    H_total = csc_matrix((D_total, D_total))

    # These are the effective weights w_i = (k * sigma_{i,c})^2
    effective_weights = [(k_param * s)**2 for s in sigma_weights]

    for i in range(4):
        if effective_weights[i] == 0:
            print(f"Skipping h_{i+1} (weight is 0)")
            continue

        print(f"Building term for h_{i+1} with weight (k*s)^2 = {effective_weights[i]:.4f}")
        h_i = h_ops[i]

        # L_i = (h_i tensor I) - (I tensor h_i)
        L_i = kron(h_i, I_N, format='csc') - kron(I_N, h_i, format='csc')

        # H_total += w_i * L_i^2
        # Note: L_i @ L_i is sparse matrix multiplication
        H_total += effective_weights[i] * (L_i @ L_i)

    print(f"Built total Hamiltonian H in {time.time() - start_time:.4f} sec.")


In [ ]:

    # --- 3. Diagonalize H ---
    start_time = time.time()
    print("Finding 2 lowest eigenvalues using sparse eigensolver (eigsh)...")

    # H is a sum of (L_i^2) terms, so it's positive semi-definite.
    # The lowest eigenvalues are the smallest algebraic ('SA') ones.
    # We ask for k=2 eigenvalues.
    try:
        # return_eigenvectors=False for speed
        eigvals = eigsh(H_total, k=2, which='SA', return_eigenvectors=False)
    except Exception as e:
        print(f"Eigenvalue computation failed: {e}")
        return np.nan

    if len(eigvals) < 2:
        print("Error: Could not find 2 eigenvalues.")
        return np.nan

    # Sort them just in case (eigsh usually does, but good practice)
    eigvals.sort()
    E0 = eigvals[0]
    E1 = eigvals[1]

    gap = E1 - E0

    print(f"Diagonalization complete in {time.time() - start_time:.4f} sec.")
    print(f"Ground State Energy E0 = {E0:.8f}")
    print(f"First Excited State Energy E1 = {E1:.8f}")

    return gap


In [ ]:

# --- Example Usage ---
if __name__ == "__main__":

    # --- Parameters ---
    N_QUBITS = 4  # Number of qubits

    # The constants sigma_{i,c}
    # [sigma_1, sigma_2, sigma_3, sigma_4]
    SIGMA_WEIGHTS = [1.0, 1.0, 1.0, 1.0]

    # The 'k' parameter
    K_PARAM = 1.0

    print("--- Exact Diagonalization for H_eff ---")
    print("WARNING: N > 8 will be very slow and consume a lot of memory.")
    if N_QUBITS > 8:
        print(f"N={N_QUBITS} is large. This may take several minutes to hours.")

    total_start = time.time()
    energy_gap = compute_hamiltonian_gap(N_QUBITS, SIGMA_WEIGHTS, K_PARAM)
    total_end = time.time()

    print("\n--- RESULT ---")
    print(f"Energy Gap (E1 - E0) = {energy_gap:.8f}")
    print(f"Total computation time: {total_end - total_start:.4f} sec.")

    # --- Example for N=6 ---
    # N_QUBITS = 6
    # SIGMA_WEIGHTS = [1.0, 0.5, 0.2, 0.1]
    # K_PARAM = 2.0
    # print("\n--- Running for N=6 ---")
    # energy_gap_6 = compute_hamiltonian_gap(N_QUBITS, SIGMA_WEIGHTS, K_PARAM)
    # print("\n--- N=6 RESULT ---")
    # print(f"Energy Gap (E1 - E0) = {energy_gap_6:.8f}")